In [250]:
import pandas as pd
import os
import sys
module_path = os.path.abspath(os.path.join('/Users/balbi/Downloads/pyMorfologik-master'))
if module_path not in sys.path:
    sys.path.append(module_path)
from pymorfologik import Morfologik
from pymorfologik.parsing import ListParser

In [236]:
def lemmatisation(text):
    parser = ListParser()
    stemmer = Morfologik()
    stemming = stemmer.stem([text], parser)
    words_list = list()
    for s in stemming:
        for i in s:
            for l in i:
                if(len(l) > 1):
                    words_list.append(l)
    return words_list

In [237]:
emotions = pd.read_csv('emotionsTable.csv')
def detectingEmotionsInText(text):
    charToRemove= str.maketrans(dict.fromkeys('!@#$?.:,'))
    words_list = lemmatisation(text)
    numberOfWords = len(words_list)
    emotionsCounter = 0
    words = pd.DataFrame()
    for el in words_list: 
        el = el.translate(charToRemove);
        data = emotions[emotions['word'] == el]
        words = words.append(data, ignore_index=True)  
    emotionsCounter = words['sentiment'].sum()
    result = emotionsCounter/numberOfWords
    return result

In [269]:
def getEmotionStatusOfText(result):
    if(result < 0):
        return 'nagatywny'
    elif(result > 0):
        return 'pozytywny'
    else:
        return 'neutralny'

In [283]:
result = detectingEmotionsInText('Uważam że zmuszanie dzieci do jedzenia warzyw jest karygodne oraz bestialskie')
print(getEmotionStatusOfText(result))

nagatywny


In [246]:
comments = pd.read_csv('oceny.csv')
numberOfComments = len(comments)
mean = comments['Ocena'].mean()

In [ ]:
%%time
comments_list = comments["Komentarz"].values
counter = 0
newCommentsTable = pd.DataFrame()
for comment in comments_list:
    test = detectingEmotionsInText(comment)
    newCommentsTable = newCommentsTable.append({'detectingEmotions': lemmatisation(comment), 'counter': test, 'result': getEmotionStatusOfText(test)}, ignore_index=True)

In [251]:
table = comments.join(newCommentsTable)
table.to_csv('FilmWeb-detectingEmotions.csv', encoding='utf-8', index=False)

CPU times: user 2.53 s, sys: 334 ms, total: 2.87 s
Wall time: 19.5 s
